In [1]:
import keras
import numpy as np
import pandas as pd

from keras.layers import Input, Dense
from keras.models import Model, Sequential

from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer

Using TensorFlow backend.


In [2]:
df = pd.read_csv("../housing-conditions-in-copenhagen/data.csv",sep="\s+")

In [3]:
df.head()

,housing,influence,contact_with_neighbours,satisfaction,n
1,1,1,1,1,21
2,1,1,1,2,21
3,1,1,1,3,28
4,1,1,2,1,14
5,1,1,2,2,19


In [4]:
df.describe()

,housing,influence,contact_with_neighbours,satisfaction,n
count,72.00000,72.000000,72.000000,72.000000,72.000000
mean,2.50000,2.000000,1.500000,2.000000,23.347222
std,1.12588,0.822226,0.503509,0.822226,17.666041
min,1.00000,1.000000,1.000000,1.000000,3.000000
25%,1.75000,1.000000,1.000000,1.000000,10.000000
50%,2.50000,2.000000,1.500000,2.000000,19.500000
75%,3.25000,3.000000,2.000000,3.000000,31.750000
max,4.00000,3.000000,2.000000,3.000000,86.000000


In [5]:
df_new = pd.concat([df,pd.get_dummies(df["housing"],prefix="housing")],axis=1).drop(["housing"],axis=1)
df_new = pd.concat([df_new,pd.get_dummies(df["influence"],prefix="influence")],axis=1).drop(["influence"],axis=1)
df_new = pd.concat([df_new,pd.get_dummies(df["satisfaction"],prefix="satisfaction")],axis=1).drop(["satisfaction"],axis=1)

In [6]:
df_new["contact_with_neighbours"] = df_new["contact_with_neighbours"].map(lambda v: 0 if v == 1 else 1)

df_new = df_new.drop(["n"])

In [7]:
df_new.head()

,contact_with_neighbours,n,housing_1,housing_2,housing_3,housing_4,influence_1,influence_2,influence_3,satisfaction_1,satisfaction_2,satisfaction_3
1,0,21,1,0,0,0,1,0,0,1,0,0
2,0,21,1,0,0,0,1,0,0,0,1,0
3,0,28,1,0,0,0,1,0,0,0,0,1
4,1,14,1,0,0,0,1,0,0,1,0,0
5,1,19,1,0,0,0,1,0,0,0,1,0


In [89]:
y = df_new["contact_with_neighbours"].values.reshape(-1,1)
X = df_new.drop(["contact_with_neighbours"], axis=1).values

In [90]:
# sc = StandardScaler()
# X = sc.fit_transform(X)

In [91]:
y.shape, X.shape

((72, 1), (72, 11))

In [92]:
indices = np.arange(72)
np.random.shuffle(indices)
num_validation_samples = int(0.2*72)

y = y[indices]
X = X[indices]

X_train = X[:-num_validation_samples]
y_train = y[:-num_validation_samples]
X_val = X[-num_validation_samples:]
y_val = y[-num_validation_samples:]

In [93]:
X_train.shape,y_train.shape,X_val.shape, y_val.shape

((58, 11), (58, 1), (14, 11), (14, 1))

In [99]:
from keras.regularizers import l1_l2
reg = l1_l2(l1=0.01, l2=0.01)

model = Sequential()
model.add(Dense(1,
                init='glorot_uniform',
                activation='sigmoid', 
                input_dim=X.shape[1]))


rmsprop = keras.optimizers.RMSprop(lr=0.01)# y = sc.fit_transform(y)
sgd = keras.optimizers.SGD(lr=0.1)
model.compile(optimizer=rmsprop, loss='binary_crossentropy')

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

model.fit(X_train, 
          y_train,
          epochs=30, 
          validation_data=(X_val, y_val),
          callbacks=[early_stopping])

/home/felipe/tf-venv3/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="glorot_uniform", activation="sigmoid", input_dim=11)`
  


Train on 58 samples, validate on 14 samples
Epoch 1/30
58/58 [==============================] - 0s - loss: 3.6872 - val_loss: 4.8987
Epoch 2/30
58/58 [==============================] - 0s - loss: 3.1931 - val_loss: 4.3503
Epoch 3/30
58/58 [==============================] - 0s - loss: 2.7570 - val_loss: 3.8347
Epoch 4/30
58/58 [==============================] - 0s - loss: 2.4010 - val_loss: 3.3106
Epoch 5/30
58/58 [==============================] - 0s - loss: 2.0814 - val_loss: 2.8140
Epoch 6/30
58/58 [==============================] - 0s - loss: 1.7635 - val_loss: 2.3486
Epoch 7/30
58/58 [==============================] - 0s - loss: 1.4866 - val_loss: 1.9665
Epoch 8/30
58/58 [==============================] - 0s - loss: 1.2881 - val_loss: 1.6689
Epoch 9/30
58/58 [==============================] - 0s - loss: 1.1107 - val_loss: 1.3813
Epoch 10/30
58/58 [==============================] - 0s - loss: 0.9732 - val_loss: 1.1473
Epoch 11/30
58/58 [==============================] - 0s - loss: 0